# Linear Statics of a Bernoulli Console with Random Geometry

A console of length 1 is created. The support is located at (0, 0, 0) and the free end is located at (1, 0, 0). The locations of points between these two follow Brownian motion, resulting in a random geometry. The console is loaded with a  concentrated force at the free end in global Z direction. The point of the example is that the support forces and moments should be the same, regardless of the path between the support and the free end.

Create the mesh:

In [27]:
import numpy as np
from polymesh import PolyData, CartesianFrame, LineData

N = 5  # the number of elements to create

rs = np.random.RandomState()

# uncomment the following line to fix seed value
#rs.seed(0)

def brownian1d(T=1, N=100, mu=0.1, sigma=0.01, S0=20):
    dt = float(T)/N
    t = np.linspace(0, T, N)
    W = rs.standard_normal(size=N)
    W = np.cumsum(W)*np.sqrt(dt)  # standard brownian motion
    X = (mu-0.5*sigma**2)*t + sigma*W
    S = S0*np.exp(X)  # geometric brownian motion
    return S

# generate points
params = dict(T=0.1, N=N, mu=0.5, sigma=0.1)
x = brownian1d(**params)
y = brownian1d(**params)
z = brownian1d(**params)

# create the mesh
frame = CartesianFrame(dim=3)
coords = np.stack([x, y, z], axis=1)
coords[:] -= coords[0]
coords = np.vstack((coords, np.array([1., 0., 0.])))
topo = np.zeros((coords.shape[0]-1, 2)).astype(int)
topo[:, 0] = np.arange(topo.shape[0])
topo[:, 1] = topo[:, 0] + 1
mesh = PolyData(coords=coords, frame=frame)
mesh['lines'] = LineData(topo=topo, frame=frame)
mesh.plot(notebook=False)

Material and load data:

In [28]:
w, h = 10., 10.  # width and height of the rectangular cross section
F = -100.  # value of the vertical load at the free end
E = 210000.0  # Young's modulus
nu = 0.3  # Poisson's ratio

# cross section
A = w * h  # area
Iy = w * h**3 / 12  # second moment of inertia around the y axis
Iz = h * w**3 / 12  # second moment of inertia around the z axis
Ix = Iy + Iz  # torsional inertia

# model stiffness matrix
G = E / (2 * (1 + nu))
Hooke = np.array([
    [E*A, 0, 0, 0],
    [0, G*Ix, 0, 0],
    [0, 0, E*Iy, 0],
    [0, 0, 0, E*Iz]
])

## Solution with 2-Node Bernoulli Element

Run a linear analysis and do some postprocessing using 2-Noded Bernoulli members.

In [29]:
from sigmaepsilon.solid import Structure, LineMesh, PointData
from sigmaepsilon.solid.fem.cells import B2 as Beam
from neumann.linalg import Vector
from polymesh.space import StandardFrame, frames_of_lines
from polymesh.space.utils import index_of_closest_point

# space
GlobalFrame = StandardFrame(dim=3)

i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
i_last = index_of_closest_point(coords, np.array([1., 0., 0.]))

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[i_last, :3] = global_load_vector
fixity[i_first, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linear_static_analysis()
reactions = structure.reaction_forces()
internal_forces = structure.internal_forces()

Check support dynams and internal dynams at the support against the actions induced by the concentrated load:

In [30]:
from neumann.logical import allclose
frame0 = StandardFrame(frames[0])
RZ = reactions[i_first, 2]
RYY = reactions[i_first, 4]
FZ = Vector(internal_forces[0, 0, :3], frame=frame0).show(GlobalFrame)[2]
FYY = Vector(internal_forces[0, 0, 3:], frame=frame0).show(GlobalFrame)[1]
assert allclose(F, FZ, atol=1e-5, rtol=1e-5)
assert allclose(F, -RZ, atol=1e-5, rtol=1e-5)
assert allclose(F, -FYY, atol=1e-5, rtol=1e-5)
assert allclose(F, RYY, atol=1e-5, rtol=1e-5)

## Solution with 3-Node Bernoulli Elements

We transform the mesh using the appropriate transformation function from `PolyMesh`.

In [31]:
from polymesh.topo.tr import L2_to_L3
coords, topo = L2_to_L3(coords, topo)

Rebuild and calculate the mesh using 3-Node elements.

In [32]:
from sigmaepsilon.solid.fem.cells import B3 as Beam

i_first = index_of_closest_point(coords, np.array([0., 0., 0.]))
i_last = index_of_closest_point(coords, np.array([1., 0., 0.]))

# support at the first, load at the last node
loads = np.zeros((coords.shape[0], 6))
fixity = np.zeros((coords.shape[0], 6)).astype(bool)
global_load_vector = Vector([0., 0, F], frame=GlobalFrame).show()
loads[i_last, :3] = global_load_vector
fixity[i_first, :] = True

# pointdata
pd = PointData(coords=coords, frame=GlobalFrame,
               loads=loads, fixity=fixity)

# celldata
frames = frames_of_lines(coords, topo)
cd = Beam(topo=topo, frames=frames)

# set up mesh and structure
mesh = LineMesh(pd, cd, model=Hooke, frame=GlobalFrame)
structure = Structure(mesh=mesh)
structure.linsolve()

# postproc
reactions = structure.reaction_forces()
internal_forces = structure.internal_forces()

Check support dynams and internal dynams at the support against the actions induced by the concentrated load:

In [33]:
frame0 = StandardFrame(frames[0])
RZ = reactions[i_first, 2]
RYY = reactions[i_first, 4]
FZ = Vector(internal_forces[0, 0, :3], frame=frame0).show(GlobalFrame)[2]
FYY = Vector(internal_forces[0, 0, 3:], frame=frame0).show(GlobalFrame)[1]
assert allclose(F, FZ, atol=1e-5, rtol=1e-5)
assert allclose(F, -RZ, atol=1e-5, rtol=1e-5)
assert allclose(F, -FYY, atol=1e-5, rtol=1e-5)
assert allclose(F, RYY, atol=1e-5, rtol=1e-5)